## S&P 500 Galaxy

Use this utility to update the returns and std_dev fields within investment-options.csv

Globals

In [1]:
# Set refresh_timeseries=True to download timeseries.  Otherwise /symbol-cache is used.
refresh_timeseries = True
throttle_limit=100
wait_time=30

In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import brownbear as bb

# format price data
pd.options.display.float_format = '{:0.2f}'.format

%matplotlib inline

In [4]:
# set size of inline plots.
'''note: rcParams can't be in same cell as import matplotlib
   or %matplotlib inline
   
   %matplotlib notebook: will lead to interactive plots embedded within
   the notebook, you can zoom and resize the figure
   
   %matplotlib inline: only draw static images in the notebook
'''
plt.rcParams["figure.figsize"] = (10, 7)

In [5]:
# read in sp500.csv
sp500 = pd.read_csv('sp500.csv')
sp500.drop(columns=['Headquarters Location', 'Date added','CIK', 'Founded'], inplace=True)
sp500.rename(columns={'Security':'Description',
                      'GICS Sector':'Asset Class'}, inplace=True)
sp500.set_index("Symbol", inplace=True)
sp500

,Description,Asset Class,GICS Sub-Industry
Symbol,,,
MMM,3M,Industrials,Industrial Conglomerates
AOS,A. O. Smith,Industrials,Building Products
ABT,Abbott Laboratories,Health Care,Health Care Equipment
ABBV,AbbVie,Health Care,Biotechnology
ACN,Accenture,Information Technology,IT Consulting & Other Services
...,...,...,...
XYL,Xylem Inc.,Industrials,Industrial Machinery & Supplies & Components
YUM,Yum! Brands,Consumer Discretionary,Restaurants
ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments


In [6]:
# read in gics-2-asset-class.csv
gics2asset_class = pd.read_csv('gics-2-asset-class.csv', skip_blank_lines=True, comment='#')
gics2asset_class.set_index("GICS", inplace=True)
gics2asset_class = gics2asset_class['Asset Class'].to_dict()
gics2asset_class

{'Energy': 'US Stocks:Energy',
 'Materials': 'US Stocks:Materials',
 'Industrials': 'US Stocks:Industrials',
 'Consumer Discretionary': 'US Stocks:Consumer Discretionary',
 'Consumer Staples': 'US Stocks:Consumer Staples',
 'Health Care': 'US Stocks:Healthcare',
 'Financials': 'US Stocks:Financials',
 'Information Technology': 'US Stocks:Technology',
 'Communication Services': 'US Stocks:Communication Services',
 'Utilities': 'US Stocks:Utilities',
 'Real Estate': 'US Stocks:Real Estate'}

In [7]:
# Map sp500 GICS sectors to brownbear defined asset classes.
def _asset_class(row):
    return gics2asset_class[row['Asset Class']]

sp500['Asset Class'] = sp500.apply(_asset_class, axis=1)

# Yahoo finance uses '-' where '.' is used in symbol names.
sp500.index = sp500.index.str.replace('.', '-', regex=False)
sp500

,Description,Asset Class,GICS Sub-Industry
Symbol,,,
MMM,3M,US Stocks:Industrials,Industrial Conglomerates
AOS,A. O. Smith,US Stocks:Industrials,Building Products
ABT,Abbott Laboratories,US Stocks:Healthcare,Health Care Equipment
ABBV,AbbVie,US Stocks:Healthcare,Biotechnology
ACN,Accenture,US Stocks:Technology,IT Consulting & Other Services
...,...,...,...
XYL,Xylem Inc.,US Stocks:Industrials,Industrial Machinery & Supplies & Components
YUM,Yum! Brands,US Stocks:Consumer Discretionary,Restaurants
ZBRA,Zebra Technologies,US Stocks:Technology,Electronic Equipment & Instruments


In [8]:
# Make symbols list.
symbols = list(sp500.index)
#symbols

In [9]:
# Get the timeseries for the symbols and compile into a single csv.
bb.fetch_timeseries(symbols, refresh=refresh_timeseries, throttle_limit=throttle_limit, wait_time=wait_time)
bb.compile_timeseries(symbols)

MMM AOS ABT ABBV ACN ADBE AMD AES AFL A APD ABNB AKAM ALB ARE ALGN ALLE LNT ALL GOOGL GOOG MO AMZN AMCR AEE AEP AXP AIG AMT AWK AMP AME AMGN APH ADI ANSS AON APA APO AAPL AMAT APTV ACGL ADM ANET AJG AIZ T ATO ADSK ADP AZO AVB AVY AXON BKR BALL BAC BAX BDX BRK-B BBY TECH BIIB BLK BX BK BA BKNG BWA BSX BMY AVGO BR BRO BF-B BLDR BG BXP CHRW CDNS CZR CPT CPB COF CAH KMX CCL CARR CAT CBOE CBRE CDW CE COR CNC CNP CF CRL SCHW 
Throttle limit reached. Waiting for 0 seconds...
CHTR CVX CMG CB CHD CI CINF CTAS CSCO C CFG CLX CME CMS KO CTSH CL CMCSA CAG COP ED STZ CEG COO CPRT GLW CPAY CTVA CSGP COST CTRA CRWD CCI CSX CMI CVS DHR DRI DVA DAY DECK DE DELL DAL DVN DXCM FANG DLR DFS DG DLTR D DPZ DOV DOW DHI DTE DUK DD EMN ETN EBAY ECL EIX EW EA ELV EMR ENPH ETR EOG EPAM EQT EFX EQIX EQR ERIE ESS EL EG EVRG ES EXC EXPE EXPD EXR XOM FFIV FDS FICO FAST FRT FDX FIS FITB FSLR FE FI FMC F 
Throttle limit reached. Waiting for 0 seconds...
FTNT FTV FOXA FOX BEN FCX GRMN IT GE GEHC GEV GEN GNRC GD GIS GM G

In [10]:
# Read symbols timeseries into a dataframe.
df = pd.read_csv('symbols-timeseries.csv', skip_blank_lines=True, comment='#')
df.set_index("Date", inplace=True)
df = df[:]
df.tail()[['NVDA', 'WST', 'ADBE', 'NOW', 'AMD', 'PYPL', 'AAPL', 'MSFT', 'DHR', 'TMO', 'AMZN', 'DXCM', 'ODFL', 'REGN', 'ROL', 'FDX', 'TMUS', 'CDNS', 'CRM', 'MSCI']]

,NVDA,WST,ADBE,NOW,AMD,PYPL,AAPL,MSFT,DHR,TMO,AMZN,DXCM,ODFL,REGN,ROL,FDX,TMUS,CDNS,CRM,MSCI
Date,,,,,,,,,,,,,,,,,,,,
2025-03-10,106.97,226.04,435.08,783.83,96.63,68.16,227.48,380.16,205.23,526.35,194.54,70.72,177.97,744.83,52.28,246.51,266.63,233.96,272.90,545.34
2025-03-11,108.75,230.77,433.66,808.40,96.76,67.98,220.84,380.45,205.61,513.77,196.59,70.26,167.03,718.16,51.91,242.96,256.67,234.25,277.05,541.72
2025-03-12,115.74,224.23,438.60,843.14,100.79,68.62,216.98,383.27,204.96,512.22,198.89,70.76,164.54,689.50,50.56,243.51,255.61,239.22,284.58,545.07
2025-03-13,115.58,223.73,377.84,811.62,98.11,66.91,209.68,378.77,204.09,508.47,193.89,68.48,162.98,680.61,50.73,240.98,259.10,241.56,271.74,541.46
2025-03-14,121.67,230.11,394.74,844.33,100.97,68.95,213.49,388.56,210.74,522.05,197.95,70.69,162.50,666.87,51.40,242.20,255.98,247.43,279.40,554.71


In [11]:
# Sample symbol.
symbol = 'MMM'

In [12]:
annual_returns = bb.annualized_returns(df, timeperiod='daily', years=1)
annual_returns[symbol]

np.float64(86.37346123681884)

In [13]:
# Calculate 1 month, 3 months, 1 year, 3 year, and 5 year annualized returns.
annual_returns_1mo = bb.annualized_returns(df, timeperiod='daily', years=1/12)
annual_returns_3mo = bb.annualized_returns(df, timeperiod='daily', years=3/12)
annual_returns_1yr = bb.annualized_returns(df, timeperiod='daily', years=1)
annual_returns_3yr = bb.annualized_returns(df, timeperiod='daily', years=3)
annual_returns_5yr = bb.annualized_returns(df, timeperiod='daily', years=5)

In [14]:
# Calculate 20 day annualized volatility.
daily_returns = df.pct_change()
years = bb.TRADING_DAYS_PER_MONTH / bb.TRADING_DAYS_PER_YEAR
vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years)
vola[symbol]

np.float64(0.29984923664055585)

In [15]:
# Calculate 20 day annualized downside volatility.
ds_vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years, downside=True)
ds_vola[symbol]

np.float64(0.20347157781308872)

In [16]:
# Resample df on a monthly basis.
df.index = pd.to_datetime(df.index)
monthly = df.resample('ME').ffill()

In [17]:
# Calculate monthly returns.
monthly_returns = monthly.pct_change()
monthly_returns[symbol]

Date
2019-01-31     NaN
2019-02-28    0.04
2019-03-31    0.00
2019-04-30   -0.09
2019-05-31   -0.15
              ... 
2024-11-30    0.04
2024-12-31   -0.03
2025-01-31    0.18
2025-02-28    0.02
2025-03-31   -0.03
Freq: ME, Name: MMM, Length: 75, dtype: float64

In [18]:
# Calculate 1 year, 3 year, and 5 year annualized standard deviation.
std_dev_1yr = bb.annualized_standard_deviation(monthly_returns, timeperiod='monthly', years=1)
std_dev_3yr = bb.annualized_standard_deviation(monthly_returns, timeperiod='monthly', years=3)
std_dev_5yr = bb.annualized_standard_deviation(monthly_returns, timeperiod='monthly', years=5)

In [19]:
# Read investment-options-header.csv
lines = []
with open('investment-options-in.csv', 'r') as f:
    lines = [line.strip() for line in f]
lines

['# Description: S&P 500 investment options. 09/29/2020',
 '',
 '# Format',
 '"Investment Option","Description","Asset Class","1 mo","3 mo","1 Yr","3 Yr","5 Yr","Vola","DS Vola","SD 1 Yr","SD 3 Yr","SD 5 Yr"']

In [20]:
# For each symbol, write out the 1 Yr, 3 Yr, 5 Yr, and std dev.
out = lines.copy()

# This is still slow (2.53 s).
for i, (index, row) in enumerate(sp500.iterrows()):

    symbol = index
    description = row['Description']
    asset_class = row['Asset Class']

    ret_1mo = annual_returns_1mo[symbol]
    ret_3mo = annual_returns_3mo[symbol]
    ret_1yr = annual_returns_1yr[symbol]
    ret_3yr = annual_returns_3yr[symbol]
    ret_5yr = annual_returns_5yr[symbol]
    
    if np.isnan(ret_3yr): ret_3yr = ret_1yr
    if np.isnan(ret_5yr): ret_5yr = ret_3yr

    _vola = vola[symbol]*100
    _ds_vola = ds_vola[symbol]*100
    sd_1yr = std_dev_1yr[symbol]*100
    sd_3yr = std_dev_3yr[symbol]*100
    sd_5yr = std_dev_5yr[symbol]*100

    out.append((
        '"{}","{}","{}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}",'
        '"{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}"'
    ).format(
        symbol, description, asset_class, ret_1mo, ret_3mo, ret_1yr, ret_3yr,
        ret_5yr, _vola, _ds_vola, sd_1yr, sd_3yr, sd_5yr
    ))

In [21]:
# Write out asset-classes.csv
with open('investment-options.csv', 'w') as f:
    for line in out:
        f.write(line + '\n')